In [849]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from datetime import datetime
import numpy as np
#import tensorflow as tf

#  Import and read the file.
supply_df = pd.read_csv("C:/Users/Elissa/UCB/dataVizfinal/SupplyData/CAISO-ALL-SUPPLY-DATA2.csv")
supply_df.head()


,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,4/10/2018,0:00:00,2018-04-10 00:00:00,1.523318e+09
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,4/10/2018,0:05:00,2018-04-10 00:05:00,1.523319e+09
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,4/10/2018,0:10:00,2018-04-10 00:10:00,1.523319e+09
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,4/10/2018,0:15:00,2018-04-10 00:15:00,1.523319e+09
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,4/10/2018,0:20:00,2018-04-10 00:20:00,1.523320e+09


In [850]:
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Large hydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Natural gas']


In [851]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

         0   1   2
0        0  00  00
1        0  05  00
2        0  10  00
3        0  15  00
4        0  20  00
...     ..  ..  ..
284793  23  35  00
284794  23  40  00
284795  23  45  00
284796  23  50  00
284797  23  55  00

[284798 rows x 3 columns]


In [852]:
#Format the baby_df into int.
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)
#Clean baby_df
baby_df= baby_df.drop([2],1)

# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})
baby_df

,Hours,Minutes
0,0,0
1,0,5
2,0,10
3,0,15
4,0,20
...,...,...
284793,23,35
284794,23,40
284795,23,45
284796,23,50


In [853]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year
#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)

baby_df.head(100)

,Hours,Minutes,Year
0,0,0,2018
1,0,5,2018
2,0,10,2018
3,0,15,2018
4,0,20,2018
...,...,...,...
95,7,55,2018
96,8,0,2018
97,8,5,2018
98,8,10,2018


In [854]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [855]:
#Make a date string column 
date_df = pd.DataFrame()
supply_df['Date1']=supply_df['Date'].astype(str)

date_df = supply_df['Date1'].str.split("-", expand=True)
date_df['DateAttribute'] = date_df[1] + date_df[2]
date_df = date_df.drop([0,1,2],1)


In [856]:
calc_df = pd.merge(baby_df, date_df, left_index=True, right_index=True, how='outer')
calc_df.head(-5)

,Hours,Minutes,Year,DemandMW,DateAttribute
0,0,0,2018,6.2356,0410
1,0,5,2018,6.2356,0410
2,0,10,2018,6.2356,0410
3,0,15,2018,6.2356,0410
4,0,20,2018,6.2356,0410
...,...,...,...,...,...
284788,23,10,2020,9.9174,1231
284789,23,15,2020,9.9174,1231
284790,23,20,2020,9.9174,1231
284791,23,25,2020,9.9174,1231


In [857]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [858]:
supply_df['Date'] = supply_df['Date'].apply(fun)
supply_df.head(1300)

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp,Sum,Date1
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,100,0:00:00,2018-04-10 00:00:00,1.523318e+09,22879.0,2018-04-10
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,100,0:05:00,2018-04-10 00:05:00,1.523319e+09,22794.0,2018-04-10
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,100,0:10:00,2018-04-10 00:10:00,1.523319e+09,22743.0,2018-04-10
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,100,0:15:00,2018-04-10 00:15:00,1.523319e+09,22488.0,2018-04-10
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,100,0:20:00,2018-04-10 00:20:00,1.523320e+09,22368.0,2018-04-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,3262.0,4010.0,3096.0,5808.0,-3.0,2253.0,7.0,0.0,105,4:15:00,2018-04-15 04:15:00,1.523766e+09,18433.0,2018-04-15
1296,3206.0,4027.0,3102.0,5786.0,0.0,2253.0,7.0,0.0,105,4:20:00,2018-04-15 04:20:00,1.523766e+09,18381.0,2018-04-15
1297,3187.0,4076.0,3106.0,5783.0,0.0,2252.0,7.0,0.0,105,4:25:00,2018-04-15 04:25:00,1.523766e+09,18411.0,2018-04-15
1298,3144.0,4018.0,3103.0,5879.0,0.0,2253.0,7.0,0.0,105,4:30:00,2018-04-15 04:30:00,1.523767e+09,18404.0,2018-04-15


In [859]:
supply_df = supply_df.drop(['Date1'],1)
final_df = pd.merge(supply_df, calc_df, left_index=True, right_index=True, how='outer')
final_df['AvgSUM(3yr)'] = 0.0
final_df['AvgSUM(3yr)'] = final_df['AvgSUM(3yr)'].astype(float)


In [860]:
df1= pd.DataFrame()
df1 = final_df.groupby(['DateAttribute','Time'])['Sum'].agg('mean')
df1 = df1.reset_index()
df1 = df1.rename(columns={df1.columns[2]: 'AvgSUM(3yr)'}).round(2)
df2 = df1.copy

print(len(df1))
df1.dtypes


105408


DateAttribute     object
Time              object
AvgSUM(3yr)      float64
dtype: object

In [861]:

df1['Date1']= "2020" + df1['DateAttribute']
df1['pDateTime'] = df1['Date1'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['pDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
ev_df2=df1.drop(['DateAttribute','Date1','pDateTime','Time'],1)

In [862]:
df1 = df1.drop(df1.index[16992:17280])
df1 = df1.reset_index(drop=True)
print(len(df1))
df1

105120


,Date2,DateAttribute,Time,AvgSUM(3yr),Date1,pDateTime
0,2020-01-01 00:00:00,0101,0:00:00,22735.50,20200101,20200101 0:00:00
1,2020-01-01 00:05:00,0101,0:05:00,22789.00,20200101,20200101 0:05:00
2,2020-01-01 00:10:00,0101,0:10:00,22778.00,20200101,20200101 0:10:00
3,2020-01-01 00:15:00,0101,0:15:00,22732.00,20200101,20200101 0:15:00
4,2020-01-01 00:20:00,0101,0:20:00,22691.50,20200101,20200101 0:20:00
...,...,...,...,...,...,...
105115,2020-12-31 23:35:00,1231,23:35:00,22745.33,20201231,20201231 23:35:00
105116,2020-12-31 23:40:00,1231,23:40:00,22671.00,20201231,20201231 23:40:00
105117,2020-12-31 23:45:00,1231,23:45:00,22602.33,20201231,20201231 23:45:00
105118,2020-12-31 23:50:00,1231,23:50:00,22541.33,20201231,20201231 23:50:00


In [863]:
df1=df1.drop(['Date2','Date1','pDateTime'],1)

df1['Date2']= "2019" + df1['DateAttribute']
df1['psDateTime'] = df1['Date2'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['psDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
ev_df1=df1.drop(['DateAttribute','psDateTime','Time'],1)
ev_df1

,Date2,AvgSUM(3yr)
0,2019-01-01 00:00:00,22735.50
1,2019-01-01 00:05:00,22789.00
2,2019-01-01 00:10:00,22778.00
3,2019-01-01 00:15:00,22732.00
4,2019-01-01 00:20:00,22691.50
...,...,...
105115,2019-12-31 23:35:00,22745.33
105116,2019-12-31 23:40:00,22671.00
105117,2019-12-31 23:45:00,22602.33
105118,2019-12-31 23:50:00,22541.33


In [864]:
df1=df1.drop(['Date2','psDateTime'],1)

df1['Date2']= "2018" + df1['DateAttribute']
df1['psDateTime'] = df1['Date2'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['psDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
ev_df=df1.drop(['DateAttribute','psDateTime','Time'],1)
ev_df

,Date2,AvgSUM(3yr)
0,2018-01-01 00:00:00,22735.50
1,2018-01-01 00:05:00,22789.00
2,2018-01-01 00:10:00,22778.00
3,2018-01-01 00:15:00,22732.00
4,2018-01-01 00:20:00,22691.50
...,...,...
105115,2018-12-31 23:35:00,22745.33
105116,2018-12-31 23:40:00,22671.00
105117,2018-12-31 23:45:00,22602.33
105118,2018-12-31 23:50:00,22541.33


In [865]:
ev_avg = ev_df.append(ev_df1)
ev_avg1 = ev_avg.append(ev_df2)
ev_avg1=ev_avg1.set_index('Date2')       
                                            
ev_avg1=ev_avg1.sort_index(ascending=True)
ev_avg1=ev_avg1.reset_index()


print(len(ev_avg1))
ev_avg1

315648


,Date2,AvgSUM(3yr)
0,2018-01-01 00:00:00,22735.50
1,2018-01-01 00:05:00,22789.00
2,2018-01-01 00:10:00,22778.00
3,2018-01-01 00:15:00,22732.00
4,2018-01-01 00:20:00,22691.50
...,...,...
315643,2020-12-31 23:35:00,22745.33
315644,2020-12-31 23:40:00,22671.00
315645,2020-12-31 23:45:00,22602.33
315646,2020-12-31 23:50:00,22541.33


In [866]:
y = ev_avg1.loc[(ev_avg1['Date2'] == '2018-04-10 00:00:00')]
print(y.index)

Int64Index([28512], dtype='int64')


In [867]:
ev_avg1 = ev_avg1.drop(ev_avg.index[0:28512])
ev_avg1 = ev_avg1.reset_index(drop=True)
ev_avg1 = ev_avg1.rename(columns={ev_avg1.columns[0]: 'DateTime'})
ev_avg1['AvgSUM(3yr)'] = ev_avg1['AvgSUM(3yr)'].astype(float)
print(len(ev_avg))

ev_avg1


210240


,DateTime,AvgSUM(3yr)
0,2018-04-10 00:00:00,21485.33
1,2018-04-10 00:05:00,21412.00
2,2018-04-10 00:10:00,21398.33
3,2018-04-10 00:15:00,21256.33
4,2018-04-10 00:20:00,21169.33
...,...,...
287131,2020-12-31 23:35:00,22745.33
287132,2020-12-31 23:40:00,22671.00
287133,2020-12-31 23:45:00,22602.33
287134,2020-12-31 23:50:00,22541.33


In [868]:
ev_avg1=ev_avg1.set_index('DateTime')
final_df = final_df.set_index('DateTime')


In [869]:
#tried a dictionary - got a bunch of NaN
final_evd = ev_avg1.to_dict()

#final_df['AvgSUM(3yr)'] = ev_avg1[ev_avg1['DateTime'].isin(final_df['DateTime'])]['AvgSUM(3yr)'].values

In [870]:
final_df['AvgSUM(3yr)'] = final_df['AvgSUM(3yr)'].map(final_evd)

final_df


,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,Unix Timestamp,Sum,Hours,Minutes,Year,DemandMW,DateAttribute,AvgSUM(3yr)
DateTime,,,,,,,,,,,,,,,,,,
2018-04-10 00:00:00,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,100,0:00:00,1.523318e+09,22879.0,0,0,2018,6.2356,0410,NaN
2018-04-10 00:05:00,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,100,0:05:00,1.523319e+09,22794.0,0,5,2018,6.2356,0410,NaN
2018-04-10 00:10:00,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,100,0:10:00,1.523319e+09,22743.0,0,10,2018,6.2356,0410,NaN
2018-04-10 00:15:00,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,100,0:15:00,1.523319e+09,22488.0,0,15,2018,6.2356,0410,NaN
2018-04-10 00:20:00,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,100,0:20:00,1.523320e+09,22368.0,0,20,2018,6.2356,0410,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 23:35:00,3183.0,8540.0,764.0,8800.0,122.0,1144.0,12.0,0.0,366,23:35:00,1.609458e+09,22565.0,23,35,2020,9.9174,1231,NaN
2020-12-31 23:40:00,3294.0,8450.0,718.0,8804.0,99.0,1144.0,12.0,0.0,366,23:40:00,1.609458e+09,22521.0,23,40,2020,9.9174,1231,NaN
2020-12-31 23:45:00,3410.0,8300.0,731.0,8725.0,90.0,1143.0,13.0,0.0,366,23:45:00,1.609458e+09,22412.0,23,45,2020,9.9174,1231,NaN
